In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Set up the notebook environment
!pip install catboost xgboost lightgbm scikit-learn
import catboost
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.io import savemat
from scipy import stats
from scipy.stats import pearsonr
from scipy import signal as sig
from scipy.signal import butter, filtfilt, welch, decimate
from scipy.interpolate import CubicSpline
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from scipy.io import loadmat
from scipy.stats import skew, kurtosis
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV



[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Tuning Coefficients Here:

In [ ]:
data_train = scipy.io.loadmat('raw_training_data.mat')
data_test = scipy.io.loadmat('leaderboard_data.mat')

In [ ]:
#window and step sizes
model_window_length = 100
model_step_size = 30

#hyperparameters
cat_iterations = 500
cat_depth = 5
cat_learning_rate = 0.05
cat_l2_leaf_reg = 4

In [ ]:
data_glove = data_train['train_dg']
data_ecog = data_train['train_ecog']
test_ecog = data_test['leaderboard_ecog']

train_dg = []
train_ecog = []

# Function to load the entire dataset as training data
def load_data(data):
    return data[0]  # Assuming each 'data' item is wrapped in an outer array

# Loop over each subject's data for glove and ECoG data
for glove_data, ecog_data in zip(data_glove, data_ecog):
    train_dg.append(load_data(glove_data))
    train_ecog.append(load_data(ecog_data))

# Now train_dg and train_ecog contain all the data for each subject as training data

# Calculate total number of samples in the training set for the first subject (as an example)
total_samples_train_ecog = len(train_ecog[0])

print(f'The total number of samples in the training set is {total_samples_train_ecog}.')


The total number of samples in the training set is 300000.


In [ ]:
ecog_test = []

for ecog_t in test_ecog:
    ecog_test.append(load_data(ecog_t))

total_samples_test_ecog = len(ecog_test[0])

print(total_samples_test_ecog)

147500


In [ ]:
def filter_data(raw_eeg, fs=1000):
    """
    Input:
      raw_eeg (samples x channels): the raw signal
      fs: the sampling rate (1000 for this dataset)

    Output:
      clean_data (samples x channels): the filtered signal
    """

    #lowcut and highcut for the bandpass filter
    lowcut = 0.15
    highcut = 200.0

    #Butterworth bandpass filter
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(4, [low, high], btype='band', analog=False)

    #apply filter to raw ECoG data
    clean_data = filtfilt(b, a, raw_eeg, axis=0)

    return clean_data


In [ ]:
#window and step sizes
window_length = model_window_length
step_size = model_step_size

#number of feature windows M for the full dataset, training, and testing sets
#M_full = 1 + (total_samples_full_ecog - window_length) // step_size
M_train = 1 + (total_samples_train_ecog - window_length) // step_size
#M_test = 1 + (total_samples_test_ecog - window_length) // step_size

#print(f'There are {M_full} windows using all the data.')
print(f'There are {M_train} windows training data.')
#print(f'There are {M_test} windows testing data.')

There are 9997 windows training data.


In [ ]:
def get_features(filtered_window, fs=1000):
    """
    Inputs:
        filtered_window (window_samples x channels): the window of the filtered ECoG signal
        fs: sampling rate

    Output:
        features (channels x num_features): the features calculated on each channel for the window
    """
    # Make sure nperseg does not exceed window size
    nperseg = min(filtered_window.shape[0], 256)

    # Basic statistical features
    mean = np.mean(filtered_window, axis=0)
    variance = np.var(filtered_window, axis=0)
    lmp = np.mean(np.abs(filtered_window), axis=0)  # Local Motor Potential

    # Compute the Power Spectral Density (PSD) for each channel
    freqs, psd = welch(filtered_window.T, fs, nperseg=nperseg)

    # Define frequency bands
    bands = {
        'alpha': (8, 12),
        #'beta': (13, 30),
        #'theta': (4, 7),  # Adding an extra band as an example
        #'low_gamma': (30, 50),  # Another example band
        'custom_band1': (18, 24),
        'custom_band2': (75, 115),
        'custom_band3': (125, 159),
        'custom_band4': (159, 175)
    }

    # Initialize a dictionary to hold the power for each band
    band_powers = {band: np.zeros(psd.shape[0]) for band in bands}

    # Calculate power for each band
    for band, (low, high) in bands.items():
        freq_mask = (freqs >= low) & (freqs <= high)
        band_powers[band] = np.sum(psd[:, freq_mask], axis=1)

    # Collect all features
    all_features = [mean, variance, lmp] + list(band_powers.values())
    features = np.vstack(all_features).T

    return features


In [ ]:
def get_features(filtered_window, fs=1000):
    """
    Inputs:
        filtered_window (window_samples x channels): the window of the filtered ECoG signal
        fs: sampling rate

    Output:
        features (channels x num_features): the features calculated on each channel for the window
    """
    # Make sure nperseg does not exceed window size
    nperseg = min(filtered_window.shape[0], 256)

    # Basic statistical features
    mean = np.mean(filtered_window, axis=0)
    variance = np.var(filtered_window, axis=0)
    lmp = np.mean(np.abs(filtered_window), axis=0)  # Local Motor Potential
    skewness = skew(filtered_window, axis=0)
    kurt = kurtosis(filtered_window, axis=0)
    ptp_amp = np.ptp(filtered_window, axis=0)  # Peak-to-peak amplitude

    # Compute the Power Spectral Density (PSD) for each channel
    freqs, psd = welch(filtered_window.T, fs, nperseg=nperseg)

    # Define frequency bands
    bands = {
        'alpha': (8, 12),
        'theta': (4, 7),
        'custom_band1': (18, 24),
        'custom_band2': (75, 115),
        'custom_band3': (125, 159),
        'custom_band4': (159, 175)
    }

    # Initialize a dictionary to hold the power for each band
    band_powers = {band: np.zeros(psd.shape[0]) for band in bands}

    # Calculate power for each band
    for band, (low, high) in bands.items():
        freq_mask = (freqs >= low) & (freqs <= high)
        band_powers[band] = np.sum(psd[:, freq_mask], axis=1)

    # Collect all features into a list
    all_features = [mean, variance, lmp, skewness, kurt, ptp_amp] + list(band_powers.values())

    # Stack features vertically and transpose to get (channels x num_features)
    features = np.vstack(all_features).T

    return features

In [ ]:
def get_windowed_feats(raw_ecog, fs, window_length_ms, window_overlap_ms):
    """
    Inputs:
        raw_ecog (samples x channels): the raw signal
        fs: the sampling rate (1000 for this dataset)
        window_length_ms: the window's length in milliseconds
        window_overlap_ms: the window's overlap in milliseconds

    Output:
        all_feats (num_windows x (channels x features)): the features for each channel for each time window
    """
    window_length = int(fs * (window_length_ms / 1000))
    window_overlap = int(fs * (window_overlap_ms / 1000))
    step_size = window_length - window_overlap

    #filter_data function placeholder
    filtered_ecog = raw_ecog

    total_samples = raw_ecog.shape[0]
    num_windows = 1 + (total_samples - window_length) // step_size

    feature_list = []

    for start_idx in tqdm(range(0, total_samples - window_length + 1, step_size)):
        end_idx = start_idx + window_length
        window = filtered_ecog[start_idx:end_idx, :]

        #features for this window
        window_features = get_features(window, fs)

        #append features to list
        feature_list.append(window_features.flatten())

    #stack to get 2D array (num_windows x features)
    all_feats = np.array(feature_list)

    return all_feats

In [ ]:
def create_R_matrix(features, N=3):
    """
    Input:
        features: The 2D array of features (num_windows x num_features_per_window).
        N: The number of previous windows to include for each prediction instance.

    Output:
        R: The response matrix.
    """
    #first N-1 rows to beginning of features matrix
    adjusted_features = np.vstack([features[:N-1], features])

    #number of total instances after adjustment
    M_prime = adjusted_features.shape[0] - (N - 1)

    #initialize response matrix R
    num_features = features.shape[1]
    R = np.zeros((M_prime, N * num_features + 1))  # +1 for the intercept term

    #fill in R
    for i in range(M_prime):
        #extract N consecutive windows of features
        consecutive_features = adjusted_features[i:i+N].flatten()
        #fill corresponding row in R, adding 1 as the last column
        R[i, 1:] = consecutive_features
        R[i, 0] = 1  # Intercept term

    return R

In [ ]:
train_ecog_1 = train_ecog[0]
train_ecog_2 = train_ecog[1]
train_ecog_3 = train_ecog[2]

train_glove_1 = train_dg[0]
train_glove_2 = train_dg[1]
train_glove_3 = train_dg[2]

In [ ]:
#feature extraction for each subject
feature_1 = get_windowed_feats(train_ecog_1, 1000, model_window_length, model_step_size)
feature_2 = get_windowed_feats(train_ecog_2, 1000, model_window_length, model_step_size)
feature_3 = get_windowed_feats(train_ecog_3, 1000, model_window_length, model_step_size)

num_target_windows = feature_1.shape[0]
num_target_windows

100%|██████████| 4285/4285 [00:03<00:00, 1232.39it/s]


4285

In [ ]:
#get R matrix
R_1 = create_R_matrix(feature_1, N=3)
R_2 = create_R_matrix(feature_2, N=3)
R_3 = create_R_matrix(feature_3, N=3)

In [ ]:
def get_windowed_target(glove_data, window_length_ms, window_overlap_ms, fs=1000):
    window_length = int(fs * (window_length_ms / 1000))
    window_overlap = int(fs * (window_overlap_ms / 1000))
    step_size = window_length - window_overlap

    num_windows = 1 + (glove_data.shape[0] - window_length) // step_size
    targets = []

    for start_idx in range(0, glove_data.shape[0] - window_length + 1, step_size):
        end_idx = start_idx + window_length
        window = glove_data[start_idx:end_idx, :]
        targets.append(np.mean(window, axis=0))  # Example: mean over the window

    return np.array(targets)

# Windowing target glove data
target_1 = get_windowed_target(train_glove_1, model_window_length, model_step_size)
target_2 = get_windowed_target(train_glove_2, model_window_length, model_step_size)
target_3 = get_windowed_target(train_glove_3, model_window_length, model_step_size)


In [ ]:
# XGBoost Model
"""xgb_model1 = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1)
xgb_model1.fit(R_1[:, 1:], target_1)

xgb_model2 = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1)
xgb_model2.fit(R_2[:, 1:], target_2)

xgb_model3 = XGBRegressor(n_estimators=300, max_depth=5, learning_rate=0.1)
xgb_model3.fit(R_3[:, 1:], target_3)

# CatBoost Model
cat_model1 = CatBoostRegressor(iterations=500, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model1.fit(R_1[:, 1:], target_1)

cat_model2 = CatBoostRegressor(iterations=500, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model2.fit(R_2[:, 1:], target_2)

cat_model3 = CatBoostRegressor(iterations=500, depth=5, learning_rate=0.1, loss_function='MultiRMSE', verbose=False)
cat_model3.fit(R_3[:, 1:], target_3)
"""

# CatBoost Model
cat_model1 = CatBoostRegressor(iterations=cat_iterations, depth=cat_depth, learning_rate=cat_learning_rate, l2_leaf_reg=cat_l2_leaf_reg, loss_function='MultiRMSE', verbose=False)
cat_model1.fit(R_1[:, 1:], target_1)

cat_model2 = CatBoostRegressor(iterations=cat_iterations, depth=cat_depth, learning_rate=cat_learning_rate, l2_leaf_reg=cat_l2_leaf_reg, loss_function='MultiRMSE', verbose=False)
cat_model2.fit(R_2[:, 1:], target_2)

cat_model3 = CatBoostRegressor(iterations=cat_iterations, depth=cat_depth, learning_rate=cat_learning_rate, l2_leaf_reg=cat_l2_leaf_reg, loss_function='MultiRMSE', verbose=False)
cat_model3.fit(R_3[:, 1:], target_3)

In [ ]:
def linear_filter(R_matrix, train_data, lambda_reg=0.01):
    num_samples_features = R_matrix.shape[0]
    num_samples_flexion = train_data.shape[0]

    # Downsample train_data if it doesn't match the number of feature windows in R_matrix
    if num_samples_features != num_samples_flexion:
        downsample_factor = num_samples_flexion // num_samples_features
        downsample_factor = max(downsample_factor, 1)  # Ensure downsample_factor at least 1
        downsize_data = decimate(train_data, downsample_factor, axis=0)

        # Ensure downsize_data has same number of samples as R_matrix
        if downsize_data.shape[0] > num_samples_features:
            downsize_data = downsize_data[:num_samples_features]
    else:
        downsize_data = train_data

    # Regularized linear filter f using matrix multiplication
    # p1 = (R^T R + lambda * I)^(-1)
    lambda_identity = lambda_reg * np.eye(R_matrix.shape[1])  # Lambda times identity matrix
    p1 = np.linalg.inv(np.matmul(R_matrix.T, R_matrix) + lambda_identity)

    # p2 = R^T Y
    p2 = np.matmul(R_matrix.T, downsize_data)

    # f = p1 * p2
    lf = np.matmul(p1, p2)

    return downsize_data, lf

In [ ]:
test_ecog_1 = ecog_test[0]
test_ecog_2 = ecog_test[1]
test_ecog_3 = ecog_test[2]

In [ ]:
#feature extraction of test set
test_feature_1 = get_windowed_feats(test_ecog_1, 1000, model_window_length, model_step_size)
test_feature_2 = get_windowed_feats(test_ecog_2, 1000, model_window_length, model_step_size)
test_feature_3 = get_windowed_feats(test_ecog_3, 1000, model_window_length, model_step_size)

100%|██████████| 2106/2106 [00:01<00:00, 1086.12it/s]


In [ ]:
#R matrix for test set
test_R_1 = create_R_matrix(test_feature_1, N=3)
test_R_2 = create_R_matrix(test_feature_2, N=3)
test_R_3 = create_R_matrix(test_feature_3, N=3)

In [ ]:
"""xgb_preds_1 = xgb_model1.predict(test_R_1[:, 1:])
xgb_preds_2 = xgb_model2.predict(test_R_2[:, 1:])
xgb_preds_3 = xgb_model3.predict(test_R_3[:, 1:])"""

cat_preds_1 = cat_model1.predict(test_R_1[:, 1:])
cat_preds_2 = cat_model2.predict(test_R_2[:, 1:])
cat_preds_3 = cat_model3.predict(test_R_3[:, 1:])

In [ ]:
"""xgb_preds_1, xgb_preds_2, xgb_preds_3"""

'xgb_preds_1, xgb_preds_2, xgb_preds_3'

In [ ]:
cat_preds_1, cat_preds_2, cat_preds_3

(array([[-0.12390067,  0.19984258, -0.2371077 , -0.52266114, -0.32721067],
        [-0.13677043,  0.04634286, -0.21680178, -0.42383306, -0.28060981],
        [-0.0956617 , -0.17081348, -0.23034635, -0.47533444, -0.21827247],
        ...,
        [-0.43642209,  0.11553463,  0.19369528,  0.84208956,  0.75963995],
        [-0.24020027,  0.20580444,  0.22073006,  0.6164415 ,  0.6731872 ],
        [-0.02575077,  0.06486145,  0.14484137,  0.83944414,  0.53978824]]),
 array([[-0.07400957, -0.0793609 ,  0.00198743, -0.2771246 , -0.00202116],
        [-0.05377738, -0.27251894, -0.16007628, -0.3203256 , -0.0996237 ],
        [-0.0961599 , -0.11811655, -0.10466072, -0.31245456, -0.0589621 ],
        ...,
        [-0.26050383,  0.07966631,  0.76838003,  0.07742264, -0.20293732],
        [ 0.01854715,  0.55421648,  0.4830742 , -0.08538168, -0.23867946],
        [-0.21323387,  0.39060166,  0.79744919,  0.33664188, -0.12273998]]),
 array([[-0.0577732 , -0.10634517, -0.22161145,  0.56377577,  0.362420

In [ ]:
from scipy.io import savemat

In [ ]:
original_length = cat_preds_1.shape[0]
desired_length = test_ecog_1.shape[0]

In [ ]:
def interpolate(prediction, original_length, desired_length):
    # Step 1: Create a time array for the original prediction
    original_time = np.linspace(0, 1, original_length)

    # Step 2: Create a time array for the desired length
    desired_time = np.linspace(0, 1, desired_length)

    # Step 3: Interpolate each column separately using cubic spline
    interpolated_prediction = np.zeros((desired_length, prediction.shape[1]))
    for i in range(prediction.shape[1]):
        cs = CubicSpline(original_time, prediction[:, i])
        interpolated_prediction[:, i] = cs(desired_time)

    return interpolated_prediction

In [ ]:
"""inter_xgb_prediction1 = interpolate(xgb_preds_1, original_length, desired_length)
inter_xgb_prediction2 = interpolate(xgb_preds_2, original_length, desired_length)
inter_xgb_prediction3 = interpolate(xgb_preds_3, original_length, desired_length)"""

'inter_xgb_prediction1 = interpolate(xgb_preds_1, original_length, desired_length)\ninter_xgb_prediction2 = interpolate(xgb_preds_2, original_length, desired_length)\ninter_xgb_prediction3 = interpolate(xgb_preds_3, original_length, desired_length)'

## **XGBoost - 0.4368**

**Training Correaltion 1.0**

([0.42975486277270897,
  0.576615967656113,
  0.14138890739763021,
  0.44750700729701787,
  0.10267421165647812],
 [0.5077436689148577,
  0.18126293860768045,
  0.21879286202665796,
  0.5181947857825426,
  0.07912924578167839],
 [0.5731551697779543,
  0.3824455186605641,
  0.46025726563488767,
  0.5024670634360416,
  0.37533030506974796])

**Training Correaltion 2.0**

([0.4100421857991916,
  0.5743259992127828,
  0.14748974360928555,
  0.44677582770641555,
  0.10947379284793216],
 [0.47727048144531226,
  0.1871533408946265,
  0.21751776062477465,
  0.5070114060467675,
  0.05319920776450609],
 [0.574777095515358,
  0.3847420304645163,
  0.47489104204454585,
  0.5214911516355769,
  0.38291076934580537])

In [ ]:
"""#XGBoost Submission
predicted_dg = np.zeros((3,1), dtype=object)
predicted_dg[0,0] = inter_xgb_prediction1
predicted_dg[1,0] = inter_xgb_prediction2
predicted_dg[2,0] = inter_xgb_prediction3

#savemat('predictions.mat', {'predicted_dg':predicted_dg})"""

"#XGBoost Submission\npredicted_dg = np.zeros((3,1), dtype=object)\npredicted_dg[0,0] = inter_xgb_prediction1\npredicted_dg[1,0] = inter_xgb_prediction2\npredicted_dg[2,0] = inter_xgb_prediction3\n\n#savemat('predictions.mat', {'predicted_dg':predicted_dg})"

In [ ]:
#print(predicted_dg)

## **CatBoost**

**Training Correaltion 1.0: n_estimators = 100**

([0.42110667057890294,
  0.5955357296984822,
  0.16680256550611047,
  0.49621501159105874,
  0.09428410688813813],
 [0.47397291001606884,
  0.18774692153335576,
  0.28783016125629185,
  0.4229055985315709,
  0.3153480826008141],
 [0.5941576953268893,
  0.3906184545554212,
  0.4958015312549799,
  0.5647895824977968,
  0.42089092458207944])

**Training Correaltion 2.0:**

([0.44241796425362817,
  0.59141093373409,
  0.17263839404211603,
  0.5097063215019813,
  0.08280680860252813],
  
  [0.4797022539095192,
  0.22737009439038464,
  0.2948164169286338,
  0.43781216948166124,
  0.31511744282831045],
 [0.5788678488123089,
  0.38464896870682586,
  0.4874284847984901,
  0.5415574332727642,
  0.4085850945940047])

**Training Correaltion 3.0: n_estimators = 300**

([0.4515281141722916,
  0.6100341432652044,
  0.17987936346454056,
  0.49870862185773174,
  0.11510920928297969],
 [0.49312217446906437,
  0.19973550922066757,
  0.28184504641469527,
  0.4870203260072296,
  0.33360122641518797],
 [0.5867612865384663,
  0.38391599391965514,
  0.516004225557485,
  0.5726448589428662,
  0.43779103958155086])

**0.4645 --> Training Correaltion 4.0: n_estimators = 500**

([0.45915435220582007,
  0.6227743996948135,
  0.18534008240262018,
  0.49759615378040606,
  0.11338631133452018],
 [0.49761276929231835,
  0.20979752269548407,
  0.28649344820015876,
  0.49269942939590744,
  0.316072076934109],
 [0.5906187000936859,
  0.38211854173786647,
  0.5084406165458811,
  0.5727193489351534,
  0.43008815127292777])

  **Training Correaltion 5.0: n_estimators = 500 + New Feature**
  ([0.4630182307903918,
  0.6074188815935773,
  0.16679204109191495,
  0.512468207157658,
  0.12314135635866383],
 [0.4912972241759117,
  0.23797972440549847,
  0.28898954817976585,
  0.49183580793905035,
  0.3356107901553058],
 [0.581519270533218,
  0.37093906808896726,
  0.49256675301902797,
  0.5683412937489525,
  0.440660179191263])

  **0.4791 --> Training Correaltion 6.0: n_estimators = 500 + New Feature + l2_leaf = 4 + step = 30**
  
  0.4313912864859705

  ([0.5522702451529398,
  0.6134971507859228,
  0.19254799634584627,
  0.5214606542371205,
  0.1108613830248457],
 [0.4867550666419851,
  0.27183218389693536,
  0.316156773032478,
  0.4971011761793195,
  0.3794104376064028],
 [0.5946865489509426,
  0.37910003999566466,
  0.5289297015821326,
  0.5780677344798992,
  0.4481922053771228])

  **Training Correaltion 7.0: New Feature + l2_leaf = 4 + step = 30 + learning rate = 0.05**

0.4376831979675297

([0.5704740105443197,
  0.6126086809542881,
  0.18570460555597088,
  0.5458383127192485,
  0.09470127594042702],
 [0.4983549761465455,
  0.23736373925260537,
  0.3243418906650215,
  0.5003635116160003,
  0.3742845292547462],
 [0.6163903123611562,
  0.3958481867275425,
  0.542169760065665,
  0.5950740108760701,
  0.47173016683333846])

In [ ]:
inter_cat_prediction1 = interpolate(cat_preds_1, original_length, desired_length)
inter_cat_prediction2 = interpolate(cat_preds_2, original_length, desired_length)
inter_cat_prediction3 = interpolate(cat_preds_3, original_length, desired_length)

In [ ]:
#CatBoost Submission
predicted_dg = np.zeros((3,1), dtype=object)
predicted_dg[0,0] = inter_cat_prediction1
predicted_dg[1,0] = inter_cat_prediction2
predicted_dg[2,0] = inter_cat_prediction3

savemat('predictions.mat', {'predicted_dg':predicted_dg})

In [ ]:
predicted_dg

array([[array([[-0.12390067,  0.19984258, -0.2371077 , -0.52266114, -0.32721067],
               [-0.12650428,  0.20034559, -0.23587323, -0.51898372, -0.32853185],
               [-0.12900892,  0.20073879, -0.23467611, -0.51538907, -0.32976895],
               ...,
               [-0.02861398,  0.07755507,  0.15567844,  0.80300038,  0.52775463],
               [-0.02714555,  0.07129202,  0.1503502 ,  0.8209137 ,  0.53359451],
               [-0.02575077,  0.06486145,  0.14484137,  0.83944414,  0.53978824]])],
       [array([[-0.07400957, -0.0793609 ,  0.00198743, -0.2771246 , -0.00202116],
               [-0.07186172, -0.08702607, -0.00254724, -0.27836727, -0.00591353],
               [-0.06978779, -0.0945142 , -0.00700777, -0.27958803, -0.00971189],
               ...,
               [-0.1961722 ,  0.4124948 ,  0.76448058,  0.30376056, -0.12886209],
               [-0.20461395,  0.4017055 ,  0.78073104,  0.32000143, -0.1258243 ],
               [-0.21323387,  0.39060166,  0.79744919, 

In [ ]:
pred = scipy.io.loadmat('predictions.mat')
print(pred['predicted_dg'])


[[array([[-0.12390067,  0.19984258, -0.2371077 , -0.52266114, -0.32721067],
         [-0.12650428,  0.20034559, -0.23587323, -0.51898372, -0.32853185],
         [-0.12900892,  0.20073879, -0.23467611, -0.51538907, -0.32976895],
         ...,
         [-0.02861398,  0.07755507,  0.15567844,  0.80300038,  0.52775463],
         [-0.02714555,  0.07129202,  0.1503502 ,  0.8209137 ,  0.53359451],
         [-0.02575077,  0.06486145,  0.14484137,  0.83944414,  0.53978824]])]
 [array([[-0.07400957, -0.0793609 ,  0.00198743, -0.2771246 , -0.00202116],
         [-0.07186172, -0.08702607, -0.00254724, -0.27836727, -0.00591353],
         [-0.06978779, -0.0945142 , -0.00700777, -0.27958803, -0.00971189],
         ...,
         [-0.1961722 ,  0.4124948 ,  0.76448058,  0.30376056, -0.12886209],
         [-0.20461395,  0.4017055 ,  0.78073104,  0.32000143, -0.1258243 ],
         [-0.21323387,  0.39060166,  0.79744919,  0.33664188, -0.12273998]])]
 [array([[-0.0577732 , -0.10634517, -0.22161145,  0.5637